In [124]:
import pandas as pd
import numpy as np
import os
import re

In [125]:
# беру путь
directory_path = r'C:\python_work\выгрузки\1_мониторинг\full_backup'

In [126]:
# названия всех файлов
file_names = os.listdir(directory_path)

In [127]:
# делаю список, в котором все файлы
all_dfs_list = [pd.read_excel(f"{directory_path}/{file_name}") for file_name in file_names]

In [128]:
# склеиваю в один
concatted_df = pd.DataFrame()
for df in all_dfs_list:
    concatted_df = pd.concat([concatted_df, df], axis=0)

In [129]:
raw = concatted_df.copy()

In [130]:
# беру все числовые колонки
all_cols = list(raw.iloc[:, :-2].columns)

In [131]:
# убираю нечисловой
all_cols.remove('47. Наличие электронной библиотечной системы (да/нет)')

In [132]:
# привожу всё к строке
raw = raw.astype('str')

In [133]:
# делаю замены
for col in all_cols:
    raw[col] = (raw[col].str.replace(',,', ',')
    .str.replace(',', '.')
    .str.replace(r'–.*)', '')
    .str.replace(r'.-', '')
    .str.replace(r'.–', '')
    .str.replace(',-', '')\
    .str.replace('проводитсяанализкорректностипредоставленныхданн.ых', '')
    .str.replace('0.0.0*', '0')
    .str.replace('отсутствуютданныевФРД.О*', '')
    .str.replace('.0.0*', '')
    .str.replace('-', '')   
    .str.replace('–', '')   
               )

In [134]:
# привожу к числовому формату
for col in all_cols:
    raw[col] = pd.to_numeric(raw[col])

In [135]:
# здесь год
raw.iloc[:, -2] = raw.iloc[:, -2].astype('int')

In [140]:
# меняю столбики местами
raw = pd.concat([
    raw.iloc[:, -1:],
    raw.iloc[:, -2:-1],
    raw.iloc[:, :-2]],axis=1)

In [141]:
# обработка столбика 'Университет'

In [142]:
raw.Университет = raw.Университет.str.replace('«', '"').str.replace('»', '"')

In [143]:
regex_name = re.compile(r'".*"')

In [144]:
uni_names = list(raw.Университет)

In [145]:
after_work = []

In [146]:
# использую regex, чтобы убрать всё за пределами кавычек
for uni in uni_names:
    try:
        match_name = regex_name.search(uni)
        after_work.append(match_name.group())
    except:
        after_work.append(uni)

In [147]:
raw.Университет = after_work

In [148]:
raw.Университет = (raw.Университет.str.replace('"', '')
                   .str.replace("федеральное государственное автономное образовательное учреждение высшего образования Первый Московский государственный медицинский университет имени И.М. Сеченова Министерства здравоохранения Российской Федерации (Сеченовский Университет)", 'Сеченовский Университет'))

In [149]:
file_name = r'C:\python_work\выгрузки\1_мониторинг\выгрузка_до_2022.xlsx'

In [150]:
raw.to_excel(file_name, index=None)